In [1]:
import pickle
import numpy as np

In [2]:
teams = ['BKN', 'MIL', 'GSW', 'LAL', 'IND', 'CHA', 'CHI', 'DET',
          'WAS', 'TOR', 'BOS', 'NYK', 'CLE', 'MEM', 'PHI', 'NOP',
          'HOU','MIN', 'ORL', 'SAS', 'OKC', 'UTA', 'SAC', 'POR',
          'DEN', 'PHX', 'DAL', 'ATL', 'MIA', 'LAC']
with open('all_teams_PBP_data.pickle', 'rb') as file:
    all_teams_stats = pickle.load(file)
epsilon = 0.05

In [3]:
for team in teams:
    team_stats = all_teams_stats[team]

    data = {game_id: dict() for game_id in team_stats}

    for game_id in team_stats:
        game_stats = team_stats[game_id]
        for player in game_stats:
            player_sum = np.array([player_stats if time > 0 else np.nan for (pie, player_stats, game_stats, time) in game_stats[player]])
            game_sum = np.array([game_stats if game_stats != 0 else np.nan for (pie, player_stats, game_stats, time) in game_stats[player]])
            pie = player_sum / game_sum
            
            data[game_id][player] = pie

    # Calculate avg
    avg_metric = dict()
    
    players = data[list(data.keys())[0]].keys() # get all players of the team

    for player in players:
        pies = []
        for game_id in team_stats:
            pie_data = data[game_id][player]
            pies.extend(list(pie_data))

        avg_metric[player] = np.nanmean(pies)

    # Convert pie to observations: 0, 1, 2
    observations = {game_id: {} for game_id in data}

    for game_id in observations:
        game_stats = data[game_id]
        for player in game_stats:
            pies = game_stats[player]
            labels = []
            for pie in pies:
                if np.isnan(pie):
                    if labels:
                        labels.append(labels[-1]) # assume the same performance as previous timestep
                    else:
                        labels.append(1) # assume avg-performance
                elif pie > avg_metric[player] + epsilon/2:
                    labels.append(2) # over-performance
                elif pie < avg_metric[player] - epsilon:
                    labels.append(0) # under-performance
                else:
                    labels.append(1) # avg-performance

            observations[game_id][player] = labels
                    
    with open(f'./observations/{team}_observations.pickle', 'wb') as file:
        pickle.dump(observations, file)
    with open(f'./average/{team}_average.pkl', 'wb') as file:
        pickle.dump(avg_metric, file)

/scratch/4284538.1.csdata/ipykernel_2921988/3961385088.py:26: RuntimeWarning: Mean of empty slice
  avg_metric[player] = np.nanmean(pies)
